# Install and Import Dependencies

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


# Import Library

In [ ]:
from datasets import load_dataset

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
import warnings
import os
import nltk

from IPython.display import Audio
from transformers import pipeline
from datasets import load_dataset
from time import time
from google.colab import drive

warnings.filterwarnings('ignore')
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
curr_path = os.getcwd()
print(curr_path)

/content


In [ ]:
!nvidia-smi

Fri May  3 07:03:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Load Dataset

In [ ]:
minds_14 = load_dataset("PolyAI/minds14", "en-US")
# to download all data for multi-lingual fine-tuning uncomment following line

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# see structure
print(minds_14)

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 563
    })
})


# Base Model

In [ ]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device

In [ ]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2Tokenizer


tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

### Inferencing

In [ ]:
wav2vec_predicted = list()
wav2vec_inference_time = list()

In [ ]:
for idx in df.index:
  # Load audio file
  audio_path = df.path[idx]
  start_time = time()

  input_audio, sr = librosa.load(audio_path, sr=16000)

  input_values = tokenizer(input_audio, return_tensors='pt').input_values
  logits = model(input_values).logits
  predicted_ids = torch.argmax(logits, dim=-1)

  text = tokenizer.batch_decode(predicted_ids)[0]

  end_time = time()
  inference_time = end_time - start_time
  wav2vec_predicted.append(text)
  wav2vec_inference_time.append(inference_time)

In [ ]:
len(wav2vec_predicted)

563

In [ ]:
df.shape

(563, 10)

In [ ]:
df['wav2vec_predicted'] = wav2vec_predicted
df['wav2vec_inference_time'] = wav2vec_inference_time
df.head()

,path,audio,transcription,english_transcription,intent_class,lang_id,intent_class_interpret,file_format,sampling_rate_ori,duration,wav2vec_predicted,wav2vec_inference_time
0,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,I would like to set up a joint account with my...,I would like to set up a joint account with my...,11,4,joint_account,wav,8000,10.837415,I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY...,I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY...
1,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,Henry County set up a joint account with my wi...,Henry County set up a joint account with my wi...,11,4,joint_account,wav,8000,6.656009,FONDERING HOW I'D SET UP A JOIN TO HELL T WITH...,FONDERING HOW I'D SET UP A JOIN TO HELL T WITH...
2,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,hi I'd like to set up a joint account with my ...,hi I'd like to set up a joint account with my ...,11,4,joint_account,wav,8000,24.234649,I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY ...,I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY ...
3,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,how do I start a joint account,how do I start a joint account,11,4,joint_account,wav,8000,3.242630,HOW DO I FURN A JOINA COUT,HOW DO I FURN A JOINA COUT
4,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,can you help me set up a joint bank account,can you help me set up a joint bank account,11,4,joint_account,wav,8000,3.840000,CAN NOW YOU HELP ME SET UP AN JOINT LEAKACCOUNT,CAN NOW YOU HELP ME SET UP AN JOINT LEAKACCOUNT


In [ ]:
df.to_csv('base_model_inferencing.csv', index=False)

In [ ]:
df['transcription'] = df['transcription'].str.lower()
df['wav2vec_predicted'] = df['wav2vec_predicted'].str.lower()

## Performance

In [ ]:
import editdistance

def calculate_wer(predicted_text, reference_text):
    distance = editdistance.eval(predicted_text.split(), reference_text.split())
    wer = distance / len(reference_text.split())
    return wer

def calculate_cer(predicted_text, reference_text):
    distance = editdistance.eval(predicted_text, reference_text)
    cer = distance / len(reference_text)
    return cer

from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

def calculate_bleu_score(predicted_text, reference_text):
    predicted_tokens = word_tokenize(predicted_text)
    reference_tokens = word_tokenize(reference_text)
    bleu_score = sentence_bleu([reference_tokens], predicted_tokens)
    return bleu_score

In [ ]:
data_metrics = df[['wav2vec_predicted', 'transcription', 'wav2vec_inference_time']]

### Word Error Rate

In [ ]:
data_metrics['wer'] = data_metrics.apply(lambda row: calculate_wer(row['wav2vec_predicted'], row['transcription']), axis=1)
print("WER:")
print(np.mean(data_metrics['wer']))

WER:
0.5245359848407404


### Character Error Rate

In [ ]:
data_metrics['cer'] = data_metrics.apply(lambda row: calculate_cer(row['wav2vec_predicted'], row['transcription']), axis=1)
print("\nCER:")
print(np.mean(data_metrics['cer']))


CER:
0.3581766993353887


### BLEU

In [ ]:
data_metrics['bleu_score'] = data_metrics.apply(lambda row: calculate_bleu_score(row['wav2vec_predicted'], row['transcription']), axis=1)
print("\nBLEU Score:")
print(np.mean(data_metrics['bleu_score']))


BLEU Score:
0.48673130572460443
